# CIFAR-10 Dataset

## Setup

In [ ]:
!wget https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz -O ./data/cifar-10-python.tar.gz

--2022-02-13 15:53:01--  https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz
Resolving www.cs.toronto.edu (www.cs.toronto.edu)... 128.100.3.30
Connecting to www.cs.toronto.edu (www.cs.toronto.edu)|128.100.3.30|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 170498071 (163M) [application/x-gzip]
Saving to: './data/cifar-10-python.tar.gz'

     0K .......... .......... .......... .......... ..........  0%  196K 14m9s
    50K .......... .......... .......... .......... ..........  0%  459K 10m6s
   100K .......... .......... .......... .......... ..........  0% 2.81M 7m3s
   150K .......... .......... .......... .......... ..........  0%  396K 7m2s
   200K .......... .......... .......... .......... ..........  0%  429K 6m55s
   250K .......... .......... .......... .......... ..........  0%  435K 6m49s
   300K .......... .......... .......... .......... ..........  0%  461K 6m42s
   350K .......... .......... .......... .......... ..........  0% 1.60M 6m5s
 

In [5]:
!tar -xzf ./data/cifar-10-python.tar.gz --directory ./data

In [1]:
!wget https://raw.githubusercontent.com/snatch59/load-cifar-10/master/load_cifar_10.py -O load_cifar_10.py

--2022-02-13 16:12:18--  https://raw.githubusercontent.com/snatch59/load-cifar-10/master/load_cifar_10.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 2606:50c0:8000::154, 2606:50c0:8003::154, 2606:50c0:8001::154, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|2606:50c0:8000::154|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4419 (4.3K) [text/plain]
Saving to: 'load_cifar_10.py'

     0K ....                                                  100%  194K=0.02s

2022-02-13 16:12:19 (194 KB/s) - 'load_cifar_10.py' saved [4419/4419]



In [4]:
import load_cifar_10

cifar_train_data, \
    cifar_train_filenames, \
    cifar_train_labels, \
    cifar_test_data, \
    cifar_test_filenames, \
    cifar_test_labels, \
    cifar_label_names = load_cifar_10.load_cifar_10_data("./data/cifar-10-batches-py")

NameError: name 'load_cifar_10_data' is not defined